### `Conv2D` code

In [1]:
import numpy as np
import torch # needed for tests
from tqdm import tqdm # needed for tests

In [2]:
class Conv2D:
    ''' Computes convolution given the input parameters '''
    
    ''' * The class implementation will be along the lines of torch.nn.Conv2D in order to 
          enable comparison of this NumPy only implementation and seamless testing
        * Can expect extensive refactoring of the existing code in the days to come
        * As part of refactoring, some code will be de-modularized
        * Old code will be retained at the end of the notebook for reference
    '''
    '''
        TODO:
        * Replace `torch.round()` with `np.allclose()` for tests
        * Add various 'same' `padding` mode
        * Implementing other features and caveats offered by nn.torch.Conv2D 
          (e.g., uniform sampling of kernel weights, bias etc.)
        * Optimizing code
    '''
    
    def __init__(
        self, 
        in_channels, 
        out_channels, 
        kernel_size, 
        padding = (0, 0), 
        stride = (1, 1), 
        dilation = (1, 1), 
        groups = 1, 
        bias = True, 
        padding_mode = 'zeros', 
        device = None, 
        dtype = None, 
        verbose = False
        ):
        super(Conv2D, self).__init__()
        
        ''' mandatory parameters '''
        self.in_channels = in_channels
        self.out_channels = out_channels
        if isinstance(kernel_size, tuple):
            self.kernel_size = kernel_size
        elif isinstance(kernel_size, int):
            self.kernel_size = (kernel_size, kernel_size)
        else:
            raise Exception('invalid input parameters: kernel_size should either be an int or a tuple')
        
        ''' optional parameters '''
        if isinstance(padding, str):
            if padding == 'valid':
                self.padding = (0, 0)
            elif padding == 'same':
                raise Exception("invalid input parameters: padding = 'same' not yet supported")
            else:
                raise Exception('invalid input parameters: padding is not valid')
        elif isinstance(padding, tuple):
            if padding[0] >= 0 and padding[1] >= 0:
                self.padding = padding
            else:
                raise Exception('invalid input parameters: padding is not valid')
        elif isinstance(padding, int):
            if padding >= 0:
                self.padding = (padding, padding)
            else:
                raise Exception('invalid input parameters: padding is not valid')
        else:
            raise Exception('invalid input parametersL padding should be either an int or a tuple')
        if isinstance(stride, tuple):
            if stride[0] >= 1 and stride[1] >= 1:
                self.stride = stride
            else:
                raise Exception('invalid input parameters: stride is not valid')
        elif isinstance(stride, int):
            if stride >= 1:
                self.stride = (stride, stride)
            else:
                raise Exception('invalid input parameters: stride is not valid')
        else:
            raise Exception('invalid input parameters: stride should be either an int or a tuple')
        if isinstance(dilation, tuple):
            if dilation[0] >= 1 and dilation[1] >= 1:
                self.dilation = dilation
            else:
                raise Exception('invalid input parameters: dilation is not valid')
        elif isinstance(dilation, int):
            if dilation >= 1:
                self.dilation = (dilation, dilation)
            else:
                raise Exception('invalid input parameters: dilation is not valid')
        else:
            raise Exception('invalid input parameters: dilation should be either an int or a tuple')
        self.groups = groups
        
        ''' optional parameters (dummy, yet to be implemented)'''
        self.bias = bias
        self.padding_mode = padding_mode
        self.device = device
        self.dtype = dtype
        
        ''' additional parameters (different from torch.nn.Conv2D)'''
        self.verbose = verbose
        self.verboseprint = print if self.verbose else lambda *a, **k: None
        self.verboseprint('*** parameters ***')
        self.verboseprint('in_channels: {}, out_channels: {}, kernel_size: {}'.format(self.in_channels, self.out_channels, self.kernel_size))
        self.verboseprint('padding: {}, stride: {}, dilation factor: {}'.format(self.padding, self.stride, self.dilation))
        self.verboseprint('groups: {}, bias: {}, padding_mode: {}, device: {}, dtype: {}'.format(self.groups, self.bias, self.padding_mode, self.device, self.dtype))
        self.verboseprint('\n')
    
    def forward(self, _input, kernels = None):
        ''' forward pass to perform convolution '''
        
        ''' do error checking '''
        _input_n, _input_c, _input_h, _input_w = _input.shape
        if _input_h + 2 * self.padding[0] < self.dilation[0] * (self.kernel_size[0] - 1) + 1: # check if (dilated) ker_h is valid
            raise Exception('invalid input parameters: kernel height is larger than input height')
        if _input_w + 2 * self.padding[1] < self.dilation[1] * (self.kernel_size[1] - 1) + 1: # check if (dilated) ker_w is valid
            raise Exception('invalid input parameters: kernel width is larger than input width')
        if ((_input_h + 2 * self.padding[0] - (self.dilation[0] * (self.kernel_size[0] - 1) + 1)) / self.stride[0]) + 1 < 0: # check if out_h is valid
            raise Exception('invalid input parameters: output height is negative')
        if ((_input_w + 2 * self.padding[1] - (self.dilation[1] * (self.kernel_size[1] - 1) + 1)) / self.stride[1]) + 1 < 0: # check if out_w is valid
            raise Exception('invalid input parameters: output width is negative')
        if  self.in_channels % self.groups != 0: # check if groups is valid
            raise Exception('invalid input parameters: input channels is not divisible by groups')
        if self.out_channels % self.groups != 0: # check if groups is valid
            raise Exception('invalid input parametes: output channels is not divisible by groups')
         
        ''' add zero padding based on the input parameters '''
        if self.padding != (0, 0):
            _input = np.array([[np.pad(channel, ((self.padding[0], self.padding[0]), (self.padding[1], self.padding[1])), 'constant', constant_values = 0) for channel in batch] for batch in _input])    
            self.verboseprint('*** padded input image ***')
            self.verboseprint('input batches: {}, input channels: {}, input height: {}, input weight: {}'.format(_input.shape[0], _input.shape[1], _input.shape[2], _input.shape[3]))
            self.verboseprint(_input)
            self.verboseprint('\n')
        
        ''' use the provided kernels or create random kernels based on the input kernel parameters '''
        if kernels is not None:
            self.verboseprint('*** kernels ***')
            self.verboseprint('kernels: {}, kernel channels: {}, kernel height: {}, kernel weight: {}'.format(self.out_channels, int(self.in_channels / self.groups), self.kernel_size[0], self.kernel_size[1]))
        else:
            kernels = []
            self.verboseprint('*** kernels ***')
            self.verboseprint('kernels: {}, kernel channels: {}, kernel height: {}, kernel weight: {}'.format(self.out_channels, int(self.in_channels / self.groups), self.kernel_size[0], self.kernel_size[1]))
            for k in range(self.out_channels):
                kernel = np.random.rand(int(self.in_channels / self.groups), self.kernel_size[0], self.kernel_size[1]) # define a random kernel based on the kernel parameters
                kernels.append(kernel)
                self.verboseprint('kernel {}'.format(k))
                self.verboseprint(kernel)
            self.verboseprint('\n')
        
        ''' dilate a kernel '''
        dil_ker_h = self.dilation[0] * (self.kernel_size[0] - 1) + 1
        dil_ker_w = self.dilation[1] * (self.kernel_size[1] - 1) + 1
        dil_kernels = []
        for kernel in kernels:
            dil_kernel = []
            for channel in kernel:
                dil_channel = np.zeros((dil_ker_h, dil_ker_w))
                for row in range(len(channel)):
                    for col in range(len(channel[0])):
                        dil_channel[self.dilation[0] * row][self.dilation[1] * col] = channel[row][col] # check if the indices 0 and 1 need to be swapped
                dil_kernel.append(dil_channel.tolist())
            dil_kernels.append(dil_kernel)
        kernels, self.kernel_size = dil_kernels, (dil_ker_h, dil_ker_w)
        self.verboseprint('*** dilated kernels ***')
        self.verboseprint('kernels: {}, dilation factor: {}, kernel channels: {}, kernel height: {}, kernel weight: {}'.format(self.out_channels, self.dilation, int(self.in_channels / self.groups), self.kernel_size[0], self.kernel_size[1]))
        for k in range(self.out_channels):
            self.verboseprint('kernel {}'.format(k))
            self.verboseprint(kernels[k])
        self.verboseprint('\n')
        
        ''' compute output volume from the input and kernel parameters '''
        _input_n, _, _input_h, _input_w = _input.shape
        out_n = int(_input_n)
        out_c = int(self.out_channels)
        out_h = int((_input_h - self.kernel_size[0]) / self.stride[0]) + 1
        out_w = int((_input_w - self.kernel_size[1]) / self.stride[1]) + 1
        output = np.zeros([out_n, out_c, out_h, out_w])
        
        ''' parse through every element of the output and compute the convolution value for that element '''
        for b_out in range(out_n):
            for c_out in range(out_c):
                for h_out in range(out_h):
                    for w_out in range(out_w):
                        # convolve kernel over the input slices
                        self.verboseprint('kernel indices, image indices')
                        self.verboseprint('[n, c, h, w]', '[n, c, h, w]')
                        convol_sum = 0
                        ker_c = int(self.in_channels / self.groups)
                        ker_h = self.kernel_size[0]
                        ker_w = self.kernel_size[1]
                        for c_ker in range(ker_c):
                            for h_ker in range(ker_h):
                                for w_ker in range(ker_w):
                                    self.verboseprint([c_out, c_ker, h_ker, w_ker], [b_out, c_ker + (ker_c * int(np.floor(c_out / (self.out_channels / self.groups)))), h_ker + self.stride[0] * h_out, w_ker + self.stride[1] * w_out])
                                    convol_sum += kernels[c_out][c_ker][h_ker][w_ker] * _input[b_out][c_ker + (ker_c * int(np.floor(c_out / (self.out_channels / self.groups))))][h_ker + self.stride[0] * h_out][w_ker + self.stride[1] * w_out] # works for all 1 <= `groups` <= in_channels
                        self.verboseprint('\n')
                        output[b_out, c_out, h_out, w_out] += convol_sum
        self.verboseprint('*** Conv2D output ***')
        output_shape = output.shape
        self.verboseprint('output batches: {}, ouput channels: {}, output height: {}, output weight: {}'.format(output_shape[0], output_shape[1], output_shape[2], output_shape[3]))
        assert((out_n, out_c, out_h, out_w) == output_shape)
        self.verboseprint(output)
        self.verboseprint('\n')
        return output

### Standalone test (random kernel, random input)

In [3]:
in_channels = 6 # input channels
out_channels = 4 # output channels
kernel_size = (2, 2) # kernel size

padding = (1, 3) # padding (optional)
stride = (2, 1) # stride (optional)
dilation = (2, 3) # dilation factor (optional)
groups = 2 # groups (optional)

in_batches = 2 # input batches
in_h = 4 # input height
in_w = 4 # input weight

_input = np.random.rand(in_batches, in_channels, in_h, in_w) # define a random image based on the input parameters
kernels = []
for k in range(out_channels):
    kernel = np.random.rand(int(in_channels / groups), kernel_size[0], kernel_size[1]) # define a random kernel based on the kernel parameters
    kernels.append(kernel)

In [4]:
# get Conv2D output with the random inputs

conv2d = Conv2D(in_channels, out_channels, kernel_size, stride = stride, padding = padding, dilation = dilation, groups = groups) # call an instance of the class with the input parameters 
_output = conv2d.forward(_input, kernels) # perform convolution
print("*** Conv2D output ***")
print(_output)

*** Conv2D output ***
[[[[0.31156689 0.12976007 0.20901492 1.13981248 0.52667357 0.84274376
    0.92394844]
   [1.6762093  1.23672141 1.6330833  3.65349577 0.7764963  0.846364
    1.53199108]]

  [[0.36683901 0.49581511 0.60500712 1.13057524 0.56241608 0.66995413
    0.82600904]
   [1.22031214 0.9447563  1.12248209 3.22654924 0.85131498 1.05977693
    1.97070397]]

  [[0.56231802 0.39599075 0.92403719 1.04508857 0.5612201  1.01598124
    0.72118227]
   [1.31288186 1.94556545 1.5148487  3.26236162 1.85659707 1.54513379
    1.68542343]]

  [[0.65862103 0.6507498  0.93873479 1.26329097 0.80511639 0.87124539
    0.79406599]
   [0.62476535 1.47360496 1.05754898 2.37225034 1.67667192 1.04871
    1.35383272]]]


 [[[0.2138944  0.32242041 0.45862936 0.7778761  0.54527008 1.03894177
    0.82270475]
   [1.22488572 1.6095512  2.43778092 2.66142763 1.27612107 1.6630827
    1.52612738]]

  [[0.45020421 0.54152677 0.90062944 1.00473857 0.62029316 1.01874124
    0.53779039]
   [1.24424514 1.65655072 

In [5]:
# get PyTorch output with the same random inputs as above

x = torch.DoubleTensor(_input)
weights = torch.stack([torch.DoubleTensor(kernel) for kernel in kernels])
output = torch.nn.functional.conv2d(x, weights, stride = stride, padding = padding, dilation = dilation, groups = groups)
print("*** PyTorch output ***")
print(output)

*** PyTorch output ***
tensor([[[[0.3116, 0.1298, 0.2090, 1.1398, 0.5267, 0.8427, 0.9239],
          [1.6762, 1.2367, 1.6331, 3.6535, 0.7765, 0.8464, 1.5320]],

         [[0.3668, 0.4958, 0.6050, 1.1306, 0.5624, 0.6700, 0.8260],
          [1.2203, 0.9448, 1.1225, 3.2265, 0.8513, 1.0598, 1.9707]],

         [[0.5623, 0.3960, 0.9240, 1.0451, 0.5612, 1.0160, 0.7212],
          [1.3129, 1.9456, 1.5148, 3.2624, 1.8566, 1.5451, 1.6854]],

         [[0.6586, 0.6507, 0.9387, 1.2633, 0.8051, 0.8712, 0.7941],
          [0.6248, 1.4736, 1.0575, 2.3723, 1.6767, 1.0487, 1.3538]]],


        [[[0.2139, 0.3224, 0.4586, 0.7779, 0.5453, 1.0389, 0.8227],
          [1.2249, 1.6096, 2.4378, 2.6614, 1.2761, 1.6631, 1.5261]],

         [[0.4502, 0.5415, 0.9006, 1.0047, 0.6203, 1.0187, 0.5378],
          [1.2442, 1.6566, 2.0034, 2.6428, 1.7455, 1.9970, 1.4123]],

         [[0.7835, 0.3092, 0.8310, 1.4363, 0.3382, 0.9727, 0.7838],
          [1.6981, 1.0028, 2.1073, 2.7804, 1.2279, 1.9266, 1.0691]],

         

In [6]:
# compare outputs of Conv2D and PyTorch
print(torch.equal(torch.round(torch.DoubleTensor(_output)), torch.round(output))) # need to round the output due to precision difference

True


### Extensive tests (random kernel, random input)

In [7]:
def valid_params(num_tests):
    ''' generates `num_tests` number of valid input and kernel parameters '''
    
    params_list = []
    sample_count = 0
    while sample_count < num_tests:
        in_channels = np.random.randint(20) + 1 # input channels
        out_channels = np.random.randint(20) + 1 # output channels
        
        kernel_h = np.random.randint(20) + 1
        kernel_w = np.random.randint(20) + 1
        kernel_size = (kernel_h, kernel_w) # kernel size
        
        padding_h = np.random.randint(10) + 1
        padding_w = np.random.randint(10) + 1
        padding = (padding_h, padding_w) # padding (optional)
        stride_h = np.random.randint(5) + 1
        stride_w = np.random.randint(5) + 1
        stride = (stride_h, stride_w) # stride (optional)
        dilation_h = np.random.randint(10) + 1
        dilation_w = np.random.randint(10) + 1
        dilation = (dilation_h, dilation_w) # dilation factor (optional)
        groups = np.random.randint(in_channels) + 1 # groups (optional)
        
        in_batches = np.random.randint(5) + 1 # input batches
        in_h = np.random.randint(30) + 5 # input height
        in_w = np.random.randint(30) + 5 # input weight
    
        ker_h_flag, ker_w_flag, out_h_flag, out_w_flag, in_group_flag, out_group_flag = True, True, True, True, True, True
        
        if in_h + 2 * padding_h < dilation_h * (kernel_h - 1) + 1: # check if (dilated) ker_h is valid
            ker_h_flag = False
        if in_w + 2 * padding_w < dilation_w * (kernel_w - 1) + 1: # check if (dilated) ker_w is valid
            ker_w_flag = False
        if ((in_h + 2 * padding_h - (dilation_h * (kernel_h - 1) + 1)) / stride_h) + 1 < 0: # check if out_h is valid
            out_h_flag = False
        if ((in_w + 2 * padding_w - (dilation_w * (kernel_w - 1) + 1)) / stride_w) + 1 < 0: # check if out_w is valid
            out_w_flag = False
        if  in_channels % groups != 0: # check if groups is valid
            in_group_flag = False
        if out_channels % groups != 0: # check if groups is valid
            out_group_flag = False
        
        if ker_h_flag and ker_w_flag and out_h_flag and out_w_flag and in_group_flag and out_group_flag:
            params_list.append({'in_channels': in_channels, 'out_channels': out_channels, 'kernel_size': kernel_size,
                          'padding': padding, 'stride': stride, 'dilation': dilation, 'groups': groups, 'in_batches': in_batches,
                          'in_h': in_h, 'in_w': in_w})
            sample_count += 1
    return params_list

In [8]:
def run_tests(num_tests):
    ''' sweep different input parameters and test by comparing outputs of Conv2D and PyTorch '''
    
    num_passed = 0
    params_list = valid_params(num_tests)
    print('Number of tests: {}\n\n'.format(len(params_list)))

    for i, params in enumerate(tqdm(params_list)):
        print('Test: {}\nParams: {}'.format(i, params))
        in_channels = params['in_channels'] # input channels
        out_channels = params['out_channels'] # output channels
        kernel_size = params['kernel_size'] # kernel size

        padding = params['padding'] # padding (optional)
        stride = params['stride'] # stride (optional)
        dilation = params['dilation'] # dilation factor (optional)
        groups = params['groups'] # groups (optional)

        in_batches = params['in_batches'] # input batches
        in_h = params['in_h'] # input height
        in_w = params['in_w'] # input weight

        _input = np.random.rand(in_batches, in_channels, in_h, in_w) # define a random image based on the input parameters
        kernels = []
        for k in range(out_channels):
            kernel = np.random.rand(int(in_channels / groups), kernel_size[0], kernel_size[1]) # define a random kernel based on the kernel parameters
            kernels.append(kernel)

        try:
            # get Conv2D output with the random inputs
            conv2d = Conv2D(in_channels, out_channels, kernel_size, stride = stride, padding = padding, dilation = dilation, groups = groups) # call an instance of the class with the input parameters 
            _output = conv2d.forward(_input, kernels) # perform convolution

            # get PyTorch output with the same random inputs as above
            x = torch.DoubleTensor(_input)
            weights = torch.stack([torch.DoubleTensor(kernel) for kernel in kernels])
            output = torch.nn.functional.conv2d(x, weights, stride = stride, padding = padding, dilation = dilation, groups = groups)

        except Exception as e:
            print(e)
            print('Result: False\n\n') # treating exception as a failed test
            continue

        # compare outputs of Conv2D and PyTorch
        result = torch.equal(torch.round(torch.DoubleTensor(_output)), torch.round(output)) # need to round the output due to precision difference
        print('Result: {}\n\n'.format(result))
        if result:
            num_passed += 1

    print('{} out of {} ({}%) tests passed'.format(num_passed, num_tests, float(100 * num_passed / num_tests)))


In [9]:
num_tests = 200
run_tests(num_tests)

Number of tests: 200




  0%|                                                   | 0/200 [00:00<?, ?it/s]

Test: 0
Params: {'in_channels': 8, 'out_channels': 1, 'kernel_size': (2, 20), 'padding': (8, 2), 'stride': (5, 3), 'dilation': (10, 1), 'groups': 1, 'in_batches': 2, 'in_h': 21, 'in_w': 30}


  0%|▏                                          | 1/200 [00:00<02:09,  1.53it/s]

Result: True


Test: 1
Params: {'in_channels': 4, 'out_channels': 12, 'kernel_size': (11, 2), 'padding': (2, 9), 'stride': (2, 4), 'dilation': (2, 7), 'groups': 2, 'in_batches': 2, 'in_h': 32, 'in_w': 25}


...

Result: True


Test: 197
Params: {'in_channels': 10, 'out_channels': 4, 'kernel_size': (10, 5), 'padding': (7, 1), 'stride': (3, 3), 'dilation': (2, 4), 'groups': 1, 'in_batches': 5, 'in_h': 23, 'in_w': 18}


100%|████████████████████████████████████████▊| 199/200 [31:45<00:03,  3.15s/it]

Result: True


Test: 198
Params: {'in_channels': 1, 'out_channels': 19, 'kernel_size': (2, 1), 'padding': (5, 3), 'stride': (2, 4), 'dilation': (4, 7), 'groups': 1, 'in_batches': 3, 'in_h': 27, 'in_w': 9}
Result: True


Test: 199
Params: {'in_channels': 2, 'out_channels': 8, 'kernel_size': (7, 6), 'padding': (7, 10), 'stride': (5, 2), 'dilation': (3, 1), 'groups': 2, 'in_batches': 5, 'in_h': 31, 'in_w': 11}


100%|█████████████████████████████████████████| 200/200 [31:46<00:00,  9.53s/it]

Result: True


200 out of 200 (100.0%) tests passed
